##### Import Required Libraries

In [5]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Import essential packages
import os
import openai
import gradio as gr
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

##### Set Up OpenAI API Key

In [16]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get OpenAI API key from environment
openai.api_key = os.getenv("OPENAI_API_KEY")

# Verify API key is loaded
if openai.api_key:
    print("OpenAI API key loaded successfully")
else:
    print("Error: OpenAI API key not found. Please add it to .env file")

OpenAI API key loaded successfully


##### Load Nestlé HR Policy Document

In [7]:
# Load the HR policy PDF document
loader = PyPDFLoader("the_nestle_hr_policy_pdf_2012.pdf")
documents = loader.load()

# Split the document into manageable chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

print(f"Total chunks created: {len(docs)}")

Total chunks created: 7


##### Create Vector Representations with FAISS

In [8]:
# Create embeddings using OpenAI
embeddings = OpenAIEmbeddings()

# Store embeddings in FAISS index
vectorstore = FAISS.from_documents(docs, embeddings)

print("FAISS vector database created successfully!")

FAISS vector database created successfully!


##### Build Question-Answering System

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# Define a prompt template with BOTH question and context
prompt_template = """
You are an AI-powered HR assistant for Nestlé.
Use the following HR policy context to answer the question.
If the answer is not in the documents, say "I cannot find relevant information in the HR policies."

Context:
{context}

Question:
{question}

Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]   # <-- include both
)

# Create a ChatOpenAI instance (wrapper around GPT-3.5 Turbo)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Build RetrievalQA chain with FAISS retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT}
)

##### Sample Test Query

In [10]:
# Test the chatbot with a sample query
test_query = "what are the advantages of joining Nestle?"
response = qa_chain.run(test_query)
print("User Question:", test_query)
print("Chatbot Answer:", response)

User Question: what are the advantages of joining Nestle?
Chatbot Answer: The advantages of joining Nestlé include good working conditions, a safe and healthy work environment, flexible employment possibilities that support a better balance of private and professional life, opportunities for personal growth and development, learning and training programs, a diverse global company culture, and the possibility to work in different countries through international assignments. Additionally, Nestlé is committed to providing attractive but realistic career moves, promotions based on sustained performance and future potential, and ensuring sustainable conditions for a gender-balanced and diverse company.


In [11]:
# Test the chatbot with a query that should be found in HR policies
test_query = "What is the maternity leave policy at Nestlé?"
response = qa_chain.run(test_query)
print("User Question:", test_query)
print("Chatbot Answer:", response)

User Question: What is the maternity leave policy at Nestlé?
Chatbot Answer: I cannot find relevant information in the HR policies.


##### Define Chatbot Function

In [12]:
# Define chatbot function for Gradio
def chatbot(query):
    response = qa_chain.run(query)
    return response

##### Build Gradio Interface

In [15]:
# Create Gradio interface
interface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(lines=3, placeholder="Ask a question about Nestlé HR policies..."),
    outputs=gr.Textbox(lines=10, label="HR Assistant Response"),
    title="Nestlé HR Assistant",
    description="An AI-powered assistant that answers questions based on Nestlé's HR policy documents."
)

# Launch the interface
interface.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
